In [ ]:
'''
Author: CLAD 23 anonymous authors
Description: Script to evaluate the RawNet2, AASIST, Res-TSSDNet and CLAD model under manipulation attacks. Please check the configuration before testing.
'''

In [ ]:
# import libraries

import torch
from torch.utils.data import Dataset, DataLoader
import yaml  # used by RawNet2 to read the configuration
import json  # used to read config file
import AASIST  # official AASIST implementation from https://github.com/clovaai/aasist/blob/main/models/AASIST.py
import os
import IPython.display as ipd  # used to display audio
from tqdm import tqdm  # progress bar
from Model import  DownStreamLinearClassifier, RawNetEncoderBaseline, RawNetBaseline, SSDNet1D  # SSDNet is the Res-TSSDNet Model
from DatasetUtils import genSpoof_list, Dataset_ASVspoof2019_train  # ASVspoof dataset utils
# Used to get the evaluation metrics
from sklearn.metrics import roc_auc_score, f1_score, balanced_accuracy_score
from evaluate_tDCF_asvspoof19 import compute_eer
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# Manipulation classes used
from DatasetUtils import VolumeChange, AddWhiteNoise, AddEnvironmentalNoise, WaveTimeStretch, AddEchoes, TimeShift, AddFade, ResampleAugmentation, pad_or_clip_batch
import torchaudio.transforms

# set random seed
torch.manual_seed(0)

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
# Evaluation Configurations
batch_size = 32
gpu = 1  # GPU id to use
torch.cuda.set_device(gpu)

# Load Config file
with open("./config.conf", "r") as f_json:
    config = json.loads(f_json.read())

In [ ]:
def load_model(model_name:str, config:dict):
    if model_name == "RawNet2":
        dir_yaml = config['rawnet2_config_path']
        with open(dir_yaml, 'r') as f_yaml:
            parser1 = yaml.safe_load(f_yaml)
        rawnet2_model_path = config['rawnet2_model_path']
        rawnet2_model = RawNetBaseline(parser1['model'], device)
        rawnet2_model = rawnet2_model.to(device)
        rawnet2_model.load_state_dict(torch.load(rawnet2_model_path,map_location=device))
        print('RawNetBaseline model loaded : {}'.format(rawnet2_model_path))
        nb_params = sum([param.view(-1).size()[0] for param in rawnet2_model.parameters()])
        print(f"Number of Rawnet2 params:{nb_params}")
        return rawnet2_model
    if model_name == "AASIST":
        with open(config['aasist_config_path'], "r") as f_json:
            aasist_config = json.loads(f_json.read())
        aasist_model_config = aasist_config["model_config"]
        aasist_model = AASIST.Model(aasist_model_config).to(device)
        nb_params = sum([param.view(-1).size()[0] for param in aasist_model.parameters()])
        print("Number of AASIST params:{}".format(nb_params))
        aasist_model.load_state_dict(
            torch.load(config['aasist_model_path'], map_location=device))
        print("Model loaded : {}".format(config["aasist_model_path"]))
        return aasist_model
    if model_name == "ResTSSDNetModel":
        res_tssdnet_model = SSDNet1D()
        check_point = torch.load(config['res_tssdnet_model_path'])
        res_tssdnet_model.load_state_dict(check_point['model_state_dict'])
        res_tssdnet_model = res_tssdnet_model.to(device)
        return res_tssdnet_model
    if model_name == "CLAD":
        with open(config['aasist_config_path'], "r") as f_json:        
            aasist_config = json.loads(f_json.read())
        aasist_model_config = aasist_config["model_config"]
        aasist_encoder = AASIST.AasistEncoder(aasist_model_config).to(device)
        downstream_model = DownStreamLinearClassifier(aasist_encoder, input_depth=160)
        checkpoint = torch.load(config['clad_model_path_for_evaluation'], map_location=device)
        downstream_model.load_state_dict(checkpoint["state_dict"])
        downstream_model = downstream_model.to(device)
        return downstream_model

In [ ]:
# Calculate the metrics, if the threshold is not given, the threshold output by the EER calculation will be used.
def get_eval_metrics(score_save_path, plot_figure=True, given_threshold=None, print_result=True):
    cm_data = np.genfromtxt(score_save_path, dtype=str)
    cm_keys = cm_data[:, 2]
    # cm_keys = 'bonafide' means 1, 'spoof' means 0
    cm_keys = np.where(cm_keys == 'bonafide', 1, 0)
    cm_scores = cm_data[:, 3].astype(float)
    # Compute EER
    # Extract bona fide (real human) and spoof scores from the CM scores
    bona_cm = cm_scores[cm_keys == 1]
    spoof_cm = cm_scores[cm_keys == 0]
    eer_cm, threshold = compute_eer(bona_cm, spoof_cm)

    auc = roc_auc_score(cm_keys, cm_scores)
    if given_threshold is not None:
        threshold = given_threshold
    y_pred = np.where(cm_scores > threshold, 1, 0)
    f1 = f1_score(cm_keys, y_pred)
    acc = balanced_accuracy_score(cm_keys, y_pred)
    # compute False Acceptance Rate and False Rejection Rate
    FAR = np.sum(cm_keys[y_pred == 1] == 0) / np.sum(cm_keys == 0)
    FRR = np.sum(cm_keys[y_pred == 0] == 1) / np.sum(cm_keys == 1)
    if print_result == True:
        print(f"EER:{eer_cm}, auc:{auc}, F1 score:{f1}, acc:{acc}, threshold:{threshold}, FAR:{FAR}, FRR:{FRR}")
    if plot_figure == True:
        # ylgnbu_pal = sns.color_palette("YlGnBu", as_cmap=True)
        sns.histplot(bona_cm, kde=False, label='Real', stat="density", element="step", fill=False, bins='auto')
        sns.histplot(spoof_cm, kde=False, label='Deepfake', stat="density",element="step", fill=False, bins='auto')
        
        plt.legend()
        plt.xlabel('Prediction score')
        plt.title('Prediction score histogram')
    return (eer_cm, auc, f1, acc, threshold, FAR, FRR)
    

In [ ]:
def evaluation_19_LA_eval(model, score_save_path, model_name, database_path, augmentations=None , batch_size = 1024, cut_length = 64600):
    # In asvspoof dataset, label = 1 means bonafide.
    model.eval()
    device = "cuda"
    # load asvspoof 2019 LA eval dataset
    d_label_trn, file_eval = genSpoof_list(dir_meta=database_path+"ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt", is_train=False, is_eval=False)
    print('no. of ASVspoof 2019 LA evaluating trials', len(file_eval))
    asvspoof_LA_eval_dataset = Dataset_ASVspoof2019_train(list_IDs=file_eval, labels=d_label_trn, base_dir=os.path.join(
        database_path+'ASVspoof2019_LA_eval/'), cut_length=cut_length)
    asvspoof_2019_LA_eval_dataloader = DataLoader(asvspoof_LA_eval_dataset, batch_size=batch_size, shuffle=False, drop_last=False, num_workers=8, pin_memory=True)  # added num_workders param to speed up.
    with open(score_save_path, 'w') as file:  # This creates an empty file or empties an existing file
        pass
    with torch.no_grad():
        for batch_idx, (audio_input,labels) in enumerate(tqdm(asvspoof_2019_LA_eval_dataloader)):
            score_list = []  
            # audio_input = torch.squeeze(audio_input)
            audio_input = audio_input.squeeze(1)          
            audio_input = audio_input.to(device)

            if augmentations != None:
                # note that some manipulation will change the length of the audio, so we need to clip or pad it to the same length
                audio_length = audio_input.shape[-1]
                # only apply the augmentation on the spoofed audio, and pad or clip it to the same length
                audio_input[labels==0] = pad_or_clip_batch(augmentations(audio_input[labels==0]), audio_length, random_clip=False)
            # check the length of the audio, if it is not the same as the cut_length, then repeat or clip it to the same length
            if audio_input.shape[-1] < cut_length:
                audio_input = audio_input.repeat(1, int(cut_length/audio_input.shape[-1])+1)[:, :cut_length]
            elif audio_input.shape[-1] > cut_length:
                audio_input = audio_input[:, :cut_length]
            
            if model_name == "ResTSSDNetModel":
                audio_input = audio_input.unsqueeze(1)  # pretrained ResTSSDNetModel takes 3D input(batch, channel, waveform), so we need to add a dimension for channels
            batch_out = model(audio_input)
            if model_name == "AASIST":
                batch_out = batch_out[1]  # the AASIST model output last_hidden_state and score and we only need the score
            # The ResTSSDNetModel output two scores, but the order is reversed. So we need to reverse it back.
            # The ResTSSDNetModel takes bonafide as 0, reference: https://github.com/ghua-ac/end-to-end-synthetic-speech-detection/blob/main/data.py#L89
            elif model_name == "ResTSSDNetModel":  
                batch_out = batch_out[:, [1,0]]
            batch_score = (batch_out[:, 1]
                          ).data.cpu().numpy().ravel()
            label_list = ['bonafide' if i==1 else 'spoof' for i in labels]
            score_list.extend(batch_score.tolist())

            with open(score_save_path, 'a+') as fh:
                for label, cm_score in zip(label_list,score_list):
                    fh.write('- - {} {}\n'.format(label, cm_score))
            fh.close()   
        print('Scores saved to {}'.format(score_save_path))
    return  get_eval_metrics(score_save_path=score_save_path, plot_figure=False)# TODO: return EER, AUC and other things when I implement

In [ ]:
evaluation_results = {}  # create a empty dict to store the results

In [ ]:
noise_dataset_path = config['noise_dataset_path']
manipulations = {
    "no_augmentation": None,
    "volume_change_50": torchaudio.transforms.Vol(gain=0.5,gain_type='amplitude'),
    "volume_change_10": torchaudio.transforms.Vol(gain=0.1,gain_type='amplitude'),
    "white_noise_15": AddWhiteNoise(max_snr_db = 15, min_snr_db=15),
    "white_noise_20": AddWhiteNoise(max_snr_db = 20, min_snr_db=20),
    "white_noise_25": AddWhiteNoise(max_snr_db = 25, min_snr_db=25),
    "env_noise_wind": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="wind", noise_dataset_path=noise_dataset_path),
    "env_noise_footsteps": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="footsteps", noise_dataset_path=noise_dataset_path),
    "env_noise_breathing": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="breathing", noise_dataset_path=noise_dataset_path),
    "env_noise_coughing": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="coughing", noise_dataset_path=noise_dataset_path),
    "env_noise_rain": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="rain", noise_dataset_path=noise_dataset_path),
    "env_noise_clock_tick": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="clock_tick", noise_dataset_path=noise_dataset_path),
    "env_noise_sneezing": AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="sneezing", noise_dataset_path=noise_dataset_path),
    "timestretch_110": WaveTimeStretch(max_ratio=1.10, min_ratio=1.10, n_fft=128),
    "timestretch_105": WaveTimeStretch(max_ratio=1.05, min_ratio=1.05, n_fft=128),
    "timestretch_095": WaveTimeStretch(max_ratio=0.95, min_ratio=0.95, n_fft=128),
    "timestretch_090": WaveTimeStretch(max_ratio=0.90, min_ratio=0.90, n_fft=128),
    "echoes_1000_02": AddEchoes(max_delay=1000, max_strengh=0.2, min_delay=1000, min_strength=0.2),
    "echoes_1000_05": AddEchoes(max_delay=1000, max_strengh=0.5, min_delay=1000, min_strength=0.5),
    "echoes_2000_05": AddEchoes(max_delay=2000, max_strengh=0.5, min_delay=2000, min_strength=0.5),
    "time_shift_1600": TimeShift(max_shift=1600, min_shift=1600),
    "time_shift_16000": TimeShift(max_shift=16000, min_shift=16000),
    "time_shift_32000": TimeShift(max_shift=32000, min_shift=32000),
    "fade_50_linear": AddFade(max_fade_size=0.5,fade_shape='linear', fix_fade_size=True),
    "fade_30_linear": AddFade(max_fade_size=0.3,fade_shape='linear', fix_fade_size=True),
    "fade_10_linear": AddFade(max_fade_size=0.1,fade_shape='linear', fix_fade_size=True),
    "fade_50_exponential": AddFade(max_fade_size=0.5,fade_shape='exponential', fix_fade_size=True),
    "fade_50_quarter_sine": AddFade(max_fade_size=0.5,fade_shape='quarter_sine', fix_fade_size=True),
    "fade_50_half_sine": AddFade(max_fade_size=0.5,fade_shape='half_sine', fix_fade_size=True),
    "fade_50_logarithmic": AddFade(max_fade_size=0.5,fade_shape='logarithmic', fix_fade_size=True),
    "resample_15000": ResampleAugmentation([15000], device="cuda"),
    "resample_15500": ResampleAugmentation([15500], device="cuda"),
    "resample_16500": ResampleAugmentation([16500], device="cuda"),
    "resample_17000": ResampleAugmentation([17000], device="cuda"),
}

In [ ]:
# Evaluate the model when the real samples are not manipulated, but the spoofed samples are manipulated.
# for model_name in ["AASIST","RawNet2","ResTSSDNetModel","CLAD"]:
for model_name in ["CLAD"]:
    if model_name == "ResTSSDNetModel":
        cut_length = 96000
    elif model_name == "RawNet2":
        cut_length = 64600  # 64000 in paper. However we found it used 64600 in the implementation(https://github.com/asvspoof-challenge/2021/blob/main/LA/Baseline-RawNet2/data_utils.py) 
    else:
        cut_length = 64600
    manipulations["env_noise_wind"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="wind", noise_dataset_path=noise_dataset_path, audio_len=cut_length)
    manipulations["env_noise_footsteps"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="footsteps", noise_dataset_path=noise_dataset_path, audio_len=cut_length)
    manipulations["env_noise_breathing"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="breathing", noise_dataset_path=noise_dataset_path, audio_len=cut_length)
    manipulations["env_noise_coughing"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="coughing", noise_dataset_path=noise_dataset_path, audio_len=cut_length)
    manipulations["env_noise_rain"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="rain", noise_dataset_path=noise_dataset_path, audio_len=cut_length)
    manipulations["env_noise_clock_tick"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="clock_tick", noise_dataset_path=noise_dataset_path, audio_len=cut_length)
    manipulations["env_noise_sneezing"] = AddEnvironmentalNoise(max_snr_db=20, min_snr_db=20, device="cuda", noise_category="sneezing", noise_dataset_path=noise_dataset_path, audio_len=cut_length)

    model = load_model(model_name,config)
    for (manipulation_name, manipulation) in manipulations.items():
        filename_prefix = model_name
        evaluation_results[manipulation_name] = evaluation_19_LA_eval(model=model, model_name=model_name, database_path=config['database_path'], batch_size = batch_size, augmentations=manipulation, score_save_path=f"scores/{filename_prefix}_{manipulation_name}_eval_19_LA_score.txt", cut_length=cut_length)
        print(f"--------{manipulation_name} finished.--------")

In [ ]:
results = {}
for model_name in ["AASIST","RawNet2","ResTSSDNetModel","CLAD"]:
    results[model_name] = {}
    # get EER threshold
    print("-----Results for model:", model_name, "-----")
    results[model_name]["no_augmentation"] = get_eval_metrics(f"scores/{model_name}_no_augmentation_eval_19_LA_score.txt", plot_figure=False, print_result=False)
    for manipulation_name in manipulations:
        results[model_name][manipulation_name] = get_eval_metrics(f"scores/{model_name}_{manipulation_name}_eval_19_LA_score.txt", plot_figure=False, given_threshold=results[model_name]["no_augmentation"][4], print_result=False)
        print(f"{manipulation_name}: EER:{results[model_name][manipulation_name][0]:.2%}, AUC:{results[model_name][manipulation_name][1]:.2%}, F1 score:{results[model_name][manipulation_name][2]:.2%}, acc:{results[model_name][manipulation_name][3]:.2%}, threshold:{results[model_name][manipulation_name][4]}, FAR:{results[model_name][manipulation_name][5]:.2%}")